In [1]:
%pip install langchain langchain_community langchain_chroma


  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached SQLAlchemy-2.0.35-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached aiohttp-3.10.5-cp312-cp312-win_amd64.whl.metadata (7.8 kB)
  Using cached langchain_text_splitters-0.3.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.5.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached chromadb-0.5.7-py3-none-any.whl.metadata (6.8 kB)
  Using cached fastapi-0.115.0-py3-none-any.whl.metadata (27 kB)
  Using cached aiohappyeyeballs-2.4.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Us

In [2]:
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


## Methods to Setup Environmental Variables for API Access

### Method 1: Enter Manually on Run (More Secure)

In [4]:
import getpass
import os

In [5]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = getpass.getpass('LANGCHAIN_API_KEY: ')
os.environ['GROQ_API_KEY'] = getpass.getpass('GROQ_API_KEY: ')

### Method 2: Setup .env file with API keys and Load

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain'

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://llamaimodel.com/requirements/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=('stats-table')
        )
    ),
)
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

In [ ]:
embedding = OllamaEmbeddings(
    model='llama3.1'
)

In [ ]:
# vectorstore.delete_collection()
# vectorstore.reset_collection()

In [ ]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(model="llama3-8b-8192")

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What's the article about?")

In [ ]:
rag_chain.invoke("what are the requirements for the 8b model?")